In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

spark = (
    SparkSession.builder
    .appName("Airlines BI Queries")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

dim_marketing = spark.read.csv("../output/Dim_Marketing_Airline.csv", header=True, inferSchema=True)
dim_operating = spark.read.csv("../output/Dim_Operating_Airline.csv", header=True, inferSchema=True)
dim_airport   = spark.read.csv("../output/Dim_Airport.csv", header=True, inferSchema=True)
dim_time      = spark.read.csv("../output/Dim_Time.csv", header=True, inferSchema=True)
dim_date      = spark.read.csv("../output/Dim_Date.csv", header=True, inferSchema=True)
fact_flight   = spark.read.csv("../output/FactFlight.csv", header=True, inferSchema=True)

fact_flight.printSchema()
dim_date.printSchema()


root
 |-- flight_id: string (nullable = true)
 |-- date_id: string (nullable = true)
 |-- departure_time_id: string (nullable = true)
 |-- arrival_time_id: string (nullable = true)
 |-- origin_airport_id: string (nullable = true)
 |-- destination_airport_id: string (nullable = true)
 |-- marketing_airline_id: string (nullable = true)
 |-- operating_airline_id: string (nullable = true)
 |-- tail_number: string (nullable = true)
 |-- dep_delay_minutes: double (nullable = true)
 |-- arr_delay_minutes: double (nullable = true)
 |-- crs_elapsed_time: double (nullable = true)
 |-- actual_elapsed_time: double (nullable = true)
 |-- distance: double (nullable = true)
 |-- status: string (nullable = true)
 |-- flight_complexity_score: double (nullable = true)

root
 |-- date_id: string (nullable = true)
 |-- flight_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_month: integer (nullable =

# 1. FILTERS

##  Які рейси були скасовані у 2020 році під час пандемії COVID-19?

In [3]:
cancelled_2020 = (
    fact_flight.alias("f")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .filter(
        (F.col("d.year") == 2020) &
        (F.col("f.status") == "cancelled")
    )
    .select(
        F.col("f.flight_id"),
        F.col("d.flight_date"),
        F.col("d.year"),
        F.col("f.origin_airport_id"),
        F.col("f.destination_airport_id"),
        F.col("f.marketing_airline_id"),
        F.col("f.operating_airline_id"),
        F.col("f.status")
    )
)

cancelled_2020.show(20, truncate=False)


+------------------------------------+-----------+----+------------------------------------+------------------------------------+------------------------------------+------------------------------------+---------+
|flight_id                           |flight_date|year|origin_airport_id                   |destination_airport_id              |marketing_airline_id                |operating_airline_id                |status   |
+------------------------------------+-----------+----+------------------------------------+------------------------------------+------------------------------------+------------------------------------+---------+
|18f4e020-9b87-4805-84bc-fead4058f23e|2020-09-15 |2020|0d6dbba4-46e2-47e2-87dc-a4777ea9c168|c23c788a-fb2d-4f67-8fa0-e38f9182ccec|0a721779-a25e-4c02-aa43-022c3d4b9ba1|27cadffb-5663-442d-9922-1e898ea08aad|cancelled|
|5b0cc1d6-58e8-4fe7-9829-d2d5cacf3a35|2020-09-16 |2020|0d6dbba4-46e2-47e2-87dc-a4777ea9c168|c23c788a-fb2d-4f67-8fa0-e38f9182ccec|0a721779-a25e-4

Пандемія мала масштабний вплив на авіаперевезення, змушуючи авіакомпанії скасовувати регулярні рейси через обмеження на подорожі та низький попит.

# 2. JOIN

## Які маркетингові авіакомпанії використовували найбільше різних операційних авіакомпаній для виконання своїх рейсів?

In [4]:
marketing_operating_usage = (
    fact_flight.alias("f")
    .join(dim_marketing.alias("m"), F.col("f.marketing_airline_id") == F.col("m.marketing_airline_id"), "inner")
    .join(dim_operating.alias("o"), F.col("f.operating_airline_id") == F.col("o.operating_airline_id"), "inner")
    .groupBy(
        F.col("m.marketing_airline_id"),
        F.col("m.iata_code"),
        F.col("m.airline_name")
    )
    .agg(
        F.countDistinct("o.operating_airline_id").alias("distinct_operating_airlines")
    )
    .orderBy(F.col("distinct_operating_airlines").desc())
)

marketing_operating_usage.show(50, truncate=False)


+------------------------------------+---------+----------------------+---------------------------+
|marketing_airline_id                |iata_code|airline_name          |distinct_operating_airlines|
+------------------------------------+---------+----------------------+---------------------------+
|0a721779-a25e-4c02-aa43-022c3d4b9ba1|AA       |American Airlines Inc.|10                         |
|15246da7-3b59-4101-b9a9-0ffddda2573a|UA       |United Air Lines Inc. |10                         |
|306ca4d6-12d2-4849-a54f-5e2b70ddd8b7|DL       |Delta Air Lines Inc.  |7                          |
|47d7408b-2072-4cd9-8852-e5a026a7a7ae|AS       |Alaska Airlines Inc.  |4                          |
|f1c889fc-f4c9-45af-8d27-a5946b0bf668|HA       |Hawaiian Airlines Inc.|2                          |
|32303bf2-c2fb-454b-b55f-eb51b871b731|VX       |Virgin America        |1                          |
|fe351a83-9393-42e1-9949-772a2299a70e|B6       |JetBlue Airways       |1                          |


Великі національні перевізники (AA, UA, DL) використовують найбільше операційних авіакомпаній

# 3. GROUP BY

## 3.1. Загальна дистанція польотів по кожній операційній авіакомпанії за рік

In [5]:
distance_by_operating_year = (
    fact_flight.alias("f")
    .join(dim_operating.alias("o"), F.col("f.operating_airline_id") == F.col("o.operating_airline_id"), "inner")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .groupBy(
        F.col("d.year"),
        F.col("o.operating_airline_id"),
        F.col("o.iata_code"),
        F.col("o.airline_name")
    )
    .agg(
        F.sum("f.distance").alias("total_distance")
    )
    .orderBy("year", "airline_name")
)

distance_by_operating_year.show(50, truncate=False)


+----+------------------------------------+---------+-----------------------------------------+--------------+
|year|operating_airline_id                |iata_code|airline_name                             |total_distance|
+----+------------------------------------+---------+-----------------------------------------+--------------+
|2018|19cb20d0-75c2-4b2f-953f-7c00201793cf|ZW       |Air Wisconsin Airlines Corp              |3.5484246E7   |
|2018|76a69608-3fd9-4de2-a801-42a9cb39af49|AS       |Alaska Airlines Inc.                     |2.41717274E8  |
|2018|f765e930-b586-48e8-8323-df113736f00e|G4       |Allegiant Air                            |8.6184621E7   |
|2018|1942ab0b-07b9-41c5-95a3-77793765be48|AA       |American Airlines Inc.                   |3.88072627E8  |
|2018|d5314902-337d-40fb-9eee-541bf9dd114c|9K       |Cape Air                                 |194790.0      |
|2018|94221ecc-8c5a-43ff-96de-60e297e081f8|PT       |Capital Cargo International              |1.130207E7    |
|

Великі числа в стовпчику total_distance представляється в науковій нотації, де:
- 1.29321953E8  = 1.29321953 × 10⁸ = 129,321,953 миль
- 1167408.0 = 1,167,408 миль

## 3.2. Середня дистанція польотів по днях тижня для кожного року

In [6]:
avg_distance_by_dow_year_names = (
    fact_flight.alias("f")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .withColumn("flight_date_dt", F.to_date(F.col("d.flight_date")))
    .withColumn("day_name", F.date_format(F.col("flight_date_dt"), "E"))  # Mon, Tue...
    .groupBy(
        F.col("d.year"),
        F.col("day_name")
    )
    .agg(
        F.avg("f.distance").alias("avg_distance")
    )
    .orderBy("year", "day_name")
)


avg_distance_by_dow_year_names.show(50, truncate=False)


+----+--------+-----------------+
|year|day_name|avg_distance     |
+----+--------+-----------------+
|2018|Fri     |783.618698052395 |
|2018|Mon     |783.796366351303 |
|2018|Sat     |817.9593560249722|
|2018|Sun     |795.4834898504482|
|2018|Thu     |784.0997093617067|
|2018|Tue     |776.1103683279575|
|2018|Wed     |777.65238170347  |
|2019|Fri     |765.1291495570922|
|2019|Mon     |764.1166638075846|
|2019|Sat     |795.2977361554483|
|2019|Sun     |774.8547581384815|
|2019|Thu     |765.1333908946879|
|2019|Tue     |757.5940629504126|
|2019|Wed     |759.1142549465316|
|2020|Fri     |752.8835846453757|
|2020|Mon     |750.4446631708912|
|2020|Sat     |772.3084566618818|
|2020|Sun     |759.2759438613554|
|2020|Thu     |753.9189688531983|
|2020|Tue     |745.4773311618973|
|2020|Wed     |748.3644932131533|
|2021|Fri     |794.4585372957931|
|2021|Mon     |791.5219779868926|
|2021|Sat     |813.3891990048804|
|2021|Sun     |798.7587976523926|
|2021|Thu     |793.6594992898291|
|2021|Tue     

Субота має найбільшу середню дистанцію (817-827 миль)
Вівторок має найменшу (745-791 миль)
Вихідні (Сб, Нд) стабільно вищі за будні

# 4. WINDOW FUNCTIONS

## 4.1. Відсоток затримок кожної авіакомпанії від загальної кількості затримок по місяцях

In [7]:
delays = (
    fact_flight.alias("f")
    .join(dim_marketing.alias("m"), F.col("f.marketing_airline_id") == F.col("m.marketing_airline_id"), "inner")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .filter(F.col("f.status") == "delayed")
    .select(
        F.col("d.year").alias("year"),
        F.col("d.month").alias("month"),
        F.col("m.marketing_airline_id").alias("marketing_airline_id"),
        F.col("m.iata_code").alias("iata_code"),
        F.col("m.airline_name").alias("airline_name")
    )
)

delays_by_airline_month = (
    delays
    .groupBy("year", "month", "marketing_airline_id", "iata_code", "airline_name")
    .agg(
        F.count("*").alias("delay_count")
    )
)

w_month = Window.partitionBy("year", "month")

delays_share = (
    delays_by_airline_month
    .withColumn("total_delays_in_month", F.sum("delay_count").over(w_month))
    .withColumn(
        "delay_pct",
        F.round(F.col("delay_count") / F.col("total_delays_in_month") * 100, 2)
    )
    .orderBy("year", "month", F.col("delay_pct").desc())
)

delays_share.show(100, truncate=False)


+----+-----+------------------------------------+---------+----------------------+-----------+---------------------+---------+
|year|month|marketing_airline_id                |iata_code|airline_name          |delay_count|total_delays_in_month|delay_pct|
+----+-----+------------------------------------+---------+----------------------+-----------+---------------------+---------+
|2018|1    |306ca4d6-12d2-4849-a54f-5e2b70ddd8b7|DL       |Delta Air Lines Inc.  |14877      |48809                |30.48    |
|2018|1    |15246da7-3b59-4101-b9a9-0ffddda2573a|UA       |United Air Lines Inc. |14086      |48809                |28.86    |
|2018|1    |c6f4efec-8c5f-42ce-9a13-a654b872cd49|WN       |Southwest Airlines Co.|8965       |48809                |18.37    |
|2018|1    |fe351a83-9393-42e1-9949-772a2299a70e|B6       |JetBlue Airways       |4884       |48809                |10.01    |
|2018|1    |47d7408b-2072-4cd9-8852-e5a026a7a7ae|AS       |Alaska Airlines Inc.  |1597       |48809            

## 4.2. Топ-3 найбільш затримані рейси для кожної авіакомпанії в кожному кварталі

In [8]:
delayed_flights = (
    fact_flight.alias("f")
    .join(dim_marketing.alias("m"), F.col("f.marketing_airline_id") == F.col("m.marketing_airline_id"), "inner")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .filter(F.col("f.status") == "delayed")
    .select(
        F.col("f.flight_id"),
        F.col("m.marketing_airline_id").alias("marketing_airline_id"),
        F.col("m.iata_code").alias("iata_code"),
        F.col("m.airline_name").alias("airline_name"),
        F.col("d.year").alias("year"),
        F.col("d.quarter").alias("quarter"),
        F.col("f.dep_delay_minutes"),
        F.col("f.arr_delay_minutes"),
        F.col("f.origin_airport_id"),
        F.col("f.destination_airport_id")
    )
)

w_airline_q = Window.partitionBy(
    "marketing_airline_id", "year", "quarter"
).orderBy(
    F.col("arr_delay_minutes").desc_nulls_last()
)

top3_delayed_per_airline_quarter = (
    delayed_flights
    .withColumn("rn", row_number().over(w_airline_q))
    .filter(F.col("rn") <= 3)
    .orderBy("airline_name", "year", "quarter", "rn")
)

top3_delayed_per_airline_quarter.show(100, truncate=False)


+------------------------------------+------------------------------------+---------+--------------------+----+-------+-----------------+-----------------+------------------------------------+------------------------------------+---+
|flight_id                           |marketing_airline_id                |iata_code|airline_name        |year|quarter|dep_delay_minutes|arr_delay_minutes|origin_airport_id                   |destination_airport_id              |rn |
+------------------------------------+------------------------------------+---------+--------------------+----+-------+-----------------+-----------------+------------------------------------+------------------------------------+---+
|90035452-b579-43e3-a938-8675233f980f|47d7408b-2072-4cd9-8852-e5a026a7a7ae|AS       |Alaska Airlines Inc.|2018|1      |692.0            |775.0            |031c7f8f-0dff-405b-ab59-a4cc88b0fcf6|86c65dfd-7fe2-418d-b727-e8df1285aa93|1  |
|8e87fa06-e93c-4dbe-bf5c-f02cfaf39249|47d7408b-2072-4cd9-8852-e5

Екстремальні затримки 10+ годин - ймовірно технічні проблеми або погода

In [9]:
spark.stop()


# Аналіз
## **Які рейси були скасовані у 2020 році під час пандемії COVID-19?**

2020 рік був періодом глобальної кризи для авіації через COVID-19 — масові обмеження, закриті аеропорти, різке падіння попиту. Аналіз скасованих рейсів дозволяє оцінити, як саме пандемія вплинула на операційну діяльність авіакомпаній.

**Бізнес-цінність:**
- Допомагає зрозуміти масштаби втрат і нестабільності у конкретних маршрутах.
- Дає змогу визначити найвразливіші напрямки або регіони (для майбутнього планування).
- Може бути використано для оцінки компенсацій, повернень коштів або логістичного планування після пандемії.

## **Які маркетингові авіакомпанії використовували найбільше різних операційних авіакомпаній для виконання своїх рейсів?**

Багато авіакомпаній працюють за моделлю code-share — коли одна компанія продає квитки, а рейс виконує інша (операційна). Це дозволяє розширювати мережу маршрутів без власного флоту.

**Бізнес-цінність:**
- Визначає, які компанії найбільше покладаються на партнерів для виконання рейсів.
- Дозволяє оцінити ефективність та ризики партнерських програм.
- Може допомогти у прийнятті рішень щодо оптимізації співпраці або власного парку літаків.

## **Яка загальна дистанція польотів по кожній операційній авіакомпанії за рік?**

Загальна відстань польотів є ключовим індикатором обсягу діяльності авіакомпанії.

**Бізнес-цінність:**
- Дає уявлення про обсяги перевезень, використання флоту та паливні витрати.
- Дозволяє виявити найактивніших перевізників на ринку.
- Є базою для оцінки вуглецевого сліду або ефективності маршрутної мережі.

## **Яка середня дистанція польотів по днях тижня для кожного року?**

Дні тижня впливають на завантаження рейсів: ділові перельоти частіше в будні, туристичні — у вихідні.

**Бізнес-цінність:**
- Дозволяє аналізувати попит і оптимізувати розклад.
- Може підказати, у які дні краще збільшувати або зменшувати кількість рейсів.
- Допомагає у плануванні персоналу, обслуговування та маркетингових кампаній (наприклад, акції на “низькі” дні).

## **Який відсоток затримок кожної авіакомпанії від загальної кількості затримок по місяцях?**

Затримки — критичний KPI для авіації, який впливає на репутацію та витрати. Відсотковий розподіл затримок по місяцях показує, хто і коли затримує рейси найчастіше.

**Бізнес-цінність:**
- Дозволяє відстежити сезонність проблем (наприклад, погода взимку або пікові навантаження влітку).
- Показує авіакомпанії з найбільшими проблемами пунктуальності.

## **Які топ-3 найбільш затримані рейси для кожної авіакомпанії в кожному кварталі?**

Замість усереднених показників — конкретика: які саме рейси створюють найбільше проблем.

**Бізнес-цінність:**
- Ідентифікує конкретні маршрути або напрямки, де найчастіше виникають затримки.
- Дає змогу приймати операційні рішення: змінити час вильоту, посилити технічну підготовку, змінити аеропорт стикування.
- Дає дані для звітів перед регуляторами або для внутрішнього контролю KPI.
